In [ ]:
from abc import ABC, abstractmethod
import pandas as pd
import numpy as np
from scipy.stats import norm
from functools import reduce
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from DateRanges import electionPeriodBoolsDF
from FinancialInstrument import FinancialInstrument

class Treasury(FinancialInstrument):
    """
    Class representing a Treasury.
    """
    def __init__(self, ticker, period, maturity, face_value = 100, frequency=2):
        """
        This is the constructor for the Treasury class. Ticker is the string of the ticker and period is an integer
        that represents which time period is wanted to be used.
            

        Args:
            ticker (string): The ticker of the Treasury.
            period (integer): The time period that should be considered.
                period = 1 -> election periods
                period = -1 -> non-election periods
                period = anything else -> total time period
            maturity (float): The maturity in years.
            face_value (float): The face value of the Treasury. Default is 100.
            frequency (integer): The number of coupon payments per year. Default is 2.
        
        Returns:
            None
        """
        self.tickerCode = ticker
        self.periodCode = period
        self.maturity = maturity
        self.face_value = face_value
        self.frequency = frequency
        self.df = pd.read_csv("../Data/TreasuryData/merged_cleaned_treasury_data.csv")
        self.df['Date'] = pd.to_datetime(self.df['Date'])
        self.ytm = self.df.set_index('Date')[[ticker + " Close"]].rename(columns={ticker + " Close": ticker + " Close YTM"}, inplace=False)
        merged_df = pd.merge(self.ytm, electionPeriodBoolsDF, left_index=True, right_index=True, how='inner')
        if period == 1:
            merged_df = merged_df[merged_df['In an Election Period']]
        elif period == -1:
            merged_df = merged_df[~merged_df['In an Election Period']]
        merged_df = merged_df.drop(columns=['In an Election Period'])
        self.ytm = self.ytm[ticker + " Close YTM"] / 100
        self.ytm = merged_df
    
    @property
    def log_returns(self):
        #To Do
        return np.log(self.ytm / self.ytm.shift(1)).dropna().rename(columns={self.tickerCode + " Close YTM": self.tickerCode + " Log Return"})

    @property
    def instrument_type(self):
        return "Treasury"
    
    @property
    def ticker(self):
        return self.tickerCode

    @property
    def period(self):
        if self.periodCode == 1:
            return "Election Periods"
        if self.periodCode == -1:
            return "Non-Election Periods"
        return "Total Time Period"


    def calculate_VaR(self, confidence_level=0.95):
        """
        This function calculates the Value at Risk (VaR) for log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the VaR calculation.
        
        Returns:
            Series with VaR of each asset.
        """
        return "To Do"
    
    def calculate_ES(self, confidence_level=0.95):
        """
        This function calculates the Expected Shortfall (ES) for the log returns at a given confidence level.

        Args:
            confidence_level (float): The confidence level for the ES calculation.
        
        Returns:
            Series with ES of each asset.
        """
        return "To Do"

treasury1 = Treasury("GB 06 GOVT 05_08_25", 0, 6)
print(treasury1.log_returns)


            GB 06 GOVT 05_08_25 Log Return
2020-11-16                        0.032435
2020-11-13                        0.021053
2020-11-12                        0.030772
2020-11-10                        0.010050
2020-11-09                       -0.072571
...                                    ...
1992-05-11                        0.002076
1992-05-08                        0.002848
1992-05-07                       -0.013273
1992-05-06                        0.008349
1992-05-05                        0.010853

[7136 rows x 1 columns]
